In [73]:
using HypothesisTests
using MultipleTesting
using CSV, DataFrames

In [128]:
df = CSV.File("diversity2.csv") |> DataFrame 

Row,Sample Name,Treatment,Tr,Observed,Chao1,Shannon
,String15,String15,String15,Int64,Float64,Float64
1,FRDC_1,no treatment,baseline,41,41.0,2.77799
2,FRDC_2,no treatment,baseline,106,106.0,3.82894
3,FRDC_5,no treatment,baseline,75,75.0,2.06959
4,FRDC_6,no treatment,baseline,125,125.0,3.67613
5,FRDC_7,no treatment,baseline,73,73.0,2.27812
6,FRDC_8,no treatment,baseline,99,99.0,2.6029
7,FRDC_19,no treatment,baseline,108,108.0,2.85988
8,FRDC_20,no treatment,baseline,80,80.0,3.41181
9,FRDC_21,no treatment,baseline,104,104.0,3.37704


In [76]:
unique(df.Tr)

4-element Vector{String15}:
 "baseline"
 "13dph"
 "16dph"
 "21dph"

In [77]:
gdfs = groupby(df, :Tr);

In [78]:
gdfs[1][:, :Treatment] |> unique

5-element Vector{String15}:
 "no treatment"
 "Control "
 "S. algae"
 "P. spongiae"
 "V. scophthalmi"

Wilcoxon rank-sum test is used to compare two independent samples,    
while Wilcoxon signed-rank test is used to compare two related samples, matched samples, or to conduct a paired difference test of repeated measurements on a single sample to assess whether their population mean ranks differ.

In [122]:
function wktest(df, treatment1, treatment2, alpha_type)
    a = filter(:Treatment => x -> x == treatment1, df)
    b = filter(:Treatment => x -> x == treatment2, df)
    #@show alpha_type; @show a[:, alpha_type]; @show b[:, alpha_type] 
    
    wt = MannWhitneyUTest(a[:, alpha_type], b[:, alpha_type]) #The Mann-Whitney U test is sometimes known as the Wilcoxon rank-sum test.
    #kt = KruskalWallisTest(a[:, alpha_type], b[:, alpha_type]) #The Kruskal-Wallis test is an extension of the Mann-Whitney U test to more than two groups.
    #@show wt; @show kt
    return wt #pvalue(kt)
end

wktest (generic function with 1 method)

In [127]:
alpha_types = ["Observed", "Chao1", "Shannon"]
[wktest(gdfs[1], "Control ", "no treatment", x) for x in alpha_types] 
# [wktest(gdfs[1], "Control ", "S. algae", x) for x in alpha_types]
# [wktest(gdfs[1], "Control ", "P. spongiae", x) for x in alpha_types]
# [wktest(gdfs[1], "Control ", "V. scophthalmi", x) for x in alpha_types]

3-element Vector{StatsAPI.HypothesisTest}:
 Approximate Mann-Whitney U test
-------------------------------
Population details:
    parameter of interest:   Location parameter (pseudomedian)
    value under h_0:         0
    point estimate:          -84.0

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           0.0081

Details:
    number of observations in each group: [3, 23]
    Mann-Whitney-U statistic:             1.0
    rank sums:                            [7.0, 344.0]
    adjustment for ties:                  6.0
    normal approximation (μ, σ):          (-33.5, 12.4578)

 Approximate Mann-Whitney U test
-------------------------------
Population details:
    parameter of interest:   Location parameter (pseudomedian)
    value under h_0:         0
    point estimate:          -84.0

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           0.0081

Details:
    number of observations in each group: [3, 23]
    

In [132]:
#[wktest(gdfs[2], "Control ", "S. algae", x) for x in alpha_types]
#[wktest(gdfs[2], "Control ", "P. spongiae", x) for x in alpha_types]
[wktest(gdfs[2], "Control ", "V. scophthalmi", x) for x in alpha_types]

3-element Vector{StatsAPI.HypothesisTest}:
 Approximate Mann-Whitney U test
-------------------------------
Population details:
    parameter of interest:   Location parameter (pseudomedian)
    value under h_0:         0
    point estimate:          28.0

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           0.3751

Details:
    number of observations in each group: [6, 6]
    Mann-Whitney-U statistic:             24.0
    rank sums:                            [45.0, 33.0]
    adjustment for ties:                  24.0
    normal approximation (μ, σ):          (6.0, 6.20117)

 Approximate Mann-Whitney U test
-------------------------------
Population details:
    parameter of interest:   Location parameter (pseudomedian)
    value under h_0:         0
    point estimate:          28.0

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           0.3751

Details:
    number of observations in each group:

In [139]:
gdfs[3]

Row,Sample Name,Treatment,Tr,Observed,Chao1,Shannon
,String15,String15,String15,Int64,Float64,Float64
1,FRDC_78,Control,16dph,77,77.0,3.0162
2,FRDC_79,Control,16dph,40,40.0,2.43655
3,FRDC_80,Control,16dph,53,53.0,2.69993
4,FRDC_81,P. spongiae,16dph,50,50.0,3.19059
5,FRDC_82,P. spongiae,16dph,64,64.0,3.07969
6,FRDC_83,P. spongiae,16dph,56,56.0,2.386
7,FRDC_84,S. algae,16dph,46,46.0,2.78636
8,FRDC_85,S. algae,16dph,68,68.0,2.8076
9,FRDC_86,S. algae,16dph,60,60.0,2.29435


In [156]:
#[wktest(gdfs[3], "Control ", "S. algae", x) for x in alpha_types]
#[wktest(gdfs[3], "Control ", "P. spongiae", x) for x in alpha_types]
[wktest(gdfs[3], "Control ", "V. scophthalmi", x) for x in alpha_types]

3-element Vector{ExactMannWhitneyUTest{Float64}}:
 Exact Mann-Whitney U test
-------------------------
Population details:
    parameter of interest:   Location parameter (pseudomedian)
    value under h_0:         0
    point estimate:          7.0

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           1.0000

Details:
    number of observations in each group: [3, 3]
    Mann-Whitney-U statistic:             4.0
    rank sums:                            [10.0, 11.0]
    adjustment for ties:                  0.0

 Exact Mann-Whitney U test
-------------------------
Population details:
    parameter of interest:   Location parameter (pseudomedian)
    value under h_0:         0
    point estimate:          7.0

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           1.0000

Details:
    number of observations in each group: [3, 3]
    Mann-Whitney-U statistic:             4.0
    rank sums:          

In [154]:
#[wktest(gdfs[4], "Control ", "S. algae", x) for x in alpha_types]
#[wktest(gdfs[4], "Control ", "P. spongiae", x) for x in alpha_types]
[wktest(gdfs[4], "Control ", "V. scophthalmi", x) for x in alpha_types]

3-element Vector{StatsAPI.HypothesisTest}:
 Approximate Mann-Whitney U test
-------------------------------
Population details:
    parameter of interest:   Location parameter (pseudomedian)
    value under h_0:         0
    point estimate:          2.0

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           0.7483

Details:
    number of observations in each group: [6, 6]
    Mann-Whitney-U statistic:             20.5
    rank sums:                            [41.5, 36.5]
    adjustment for ties:                  6.0
    normal approximation (μ, σ):          (2.5, 6.23407)

 Approximate Mann-Whitney U test
-------------------------------
Population details:
    parameter of interest:   Location parameter (pseudomedian)
    value under h_0:         0
    point estimate:          2.0

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           0.7483

Details:
    number of observations in each group: [6

In [121]:
a = filter(:Treatment => x -> x == "Control ", gdfs[1]) 
b = filter(:Treatment => x -> x == "no treatment", gdfs[1]) 
c = filter(:Treatment => x -> x == "S. algae", gdfs[1])
d = filter(:Treatment => x -> x == "P. spongiae", gdfs[1])
e = filter(:Treatment => x -> x == "V. scophthalmi", gdfs[1])

KruskalWallisTest(a.Observed, b.Observed, c.Observed, d.Observed, e.Observed)
#KruskalWallisTest(a.Chao1, b.Chao1, c.Chao1, d.Chao1, e.Chao1)
#KruskalWallisTest(a.Shannon, b.Shannon, c.Shannon, d.Shannon, e.Shannon)

Kruskal-Wallis rank sum test (chi-square approximation)
-------------------------------------------------------
Population details:
    parameter of interest:   Location parameters
    value under h_0:         "all equal"
    point estimate:          NaN

Test summary:
    outcome with 95% confidence: reject h_0
    one-sided p-value:           0.0003

Details:
    number of observation in each group: [3, 23, 3, 3, 3]
    χ²-statistic:                        21.2179
    rank sums:                           [21.0, 546.0, 18.0, 27.0, 18.0]
    degrees of freedom:                  4
    adjustment for ties:                 0.99986


In [120]:
a = filter(:Treatment => x -> x == "Control ", gdfs[2]) 
c = filter(:Treatment => x -> x == "S. algae", gdfs[2])
d = filter(:Treatment => x -> x == "P. spongiae", gdfs[2])
e = filter(:Treatment => x -> x == "V. scophthalmi", gdfs[2])

#KruskalWallisTest(a.Observed, c.Observed, d.Observed, e.Observed)
#KruskalWallisTest(a.Chao1, c.Chao1, d.Chao1, e.Chao1)
KruskalWallisTest(a.Shannon, c.Shannon, d.Shannon, e.Shannon)

Kruskal-Wallis rank sum test (chi-square approximation)
-------------------------------------------------------
Population details:
    parameter of interest:   Location parameters
    value under h_0:         "all equal"
    point estimate:          NaN

Test summary:
    outcome with 95% confidence: reject h_0
    one-sided p-value:           0.0071

Details:
    number of observation in each group: [6, 6, 6, 6]
    χ²-statistic:                        12.0867
    rank sums:                           [65.0, 42.0, 124.0, 69.0]
    degrees of freedom:                  3
    adjustment for ties:                 1.0


In [146]:
a = filter(:Treatment => x -> x == "Control ", gdfs[3]) 
c = filter(:Treatment => x -> x == "S. algae", gdfs[3])
d = filter(:Treatment => x -> x == "P. spongiae", gdfs[3])
e = filter(:Treatment => x -> x == "V. scophthalmi", gdfs[3])

#KruskalWallisTest(a.Observed, c.Observed, d.Observed, e.Observed)
#KruskalWallisTest(a.Chao1, c.Chao1, d.Chao1, e.Chao1)
KruskalWallisTest(a.Shannon, c.Shannon, d.Shannon, e.Shannon)

Kruskal-Wallis rank sum test (chi-square approximation)
-------------------------------------------------------
Population details:
    parameter of interest:   Location parameters
    value under h_0:         "all equal"
    point estimate:          NaN

Test summary:
    outcome with 95% confidence: fail to reject h_0
    one-sided p-value:           0.7641

Details:
    number of observation in each group: [3, 3, 3, 3]
    χ²-statistic:                        1.15385
    rank sums:                           [19.0, 16.0, 25.0, 18.0]
    degrees of freedom:                  3
    adjustment for ties:                 1.0


In [159]:
a = filter(:Treatment => x -> x == "Control ", gdfs[4]) 
c = filter(:Treatment => x -> x == "S. algae", gdfs[4])
d = filter(:Treatment => x -> x == "P. spongiae", gdfs[4])
e = filter(:Treatment => x -> x == "V. scophthalmi", gdfs[4])

#KruskalWallisTest(a.Observed, c.Observed, d.Observed, e.Observed)
#KruskalWallisTest(a.Chao1, c.Chao1, d.Chao1, e.Chao1)
KruskalWallisTest(a.Shannon, c.Shannon, d.Shannon, e.Shannon)

Kruskal-Wallis rank sum test (chi-square approximation)
-------------------------------------------------------
Population details:
    parameter of interest:   Location parameters
    value under h_0:         "all equal"
    point estimate:          NaN

Test summary:
    outcome with 95% confidence: fail to reject h_0
    one-sided p-value:           0.2120

Details:
    number of observation in each group: [6, 6, 5, 6]
    χ²-statistic:                        4.5029
    rank sums:                           [91.0, 87.0, 48.0, 50.0]
    degrees of freedom:                  3
    adjustment for ties:                 1.0


In [161]:
p = [0.0714, 0.9333, 1.0000, 0.0823]
p = [0.5714, 0.6667, 1.0000, 0.0173]
p = [0.0098, 0.0333, 0.3471, 0.3894, 0.0301, 0.7119]
p = [0.5338, 0.4698, 0.9452, 0.3660, 0.2403, 0.4286]
p = [0.0159, 0.1349, 0.5813, 0.1270, 0.5238, 0.0714, 1.0000]
#p = [1.0000, 0.1508, 0.7922, 0.8413, 0.2222, 0.0952, 0.0952]
pvals = PValues(p)
adjust(pvals, BenjaminiHochberg())

7-element Vector{Float64}:
 0.11130000000000001
 0.23607499999999998
 0.6781833333333335
 0.23607499999999998
 0.6781833333333335
 0.23607499999999998
 1.0

In [174]:
p = [0.0081, 1.0000, 1.0000, 1.0000]
p = [0.0177, 0.7000, 0.7000, 1.0000]
p = [0.1460, 0.1249, 0.3751]
p = [0.2403, 0.0087, 0.8182]
p = [1.0000, 1.0000, 1.0000]
p = [1.0000, 0.7000, 1.0000]
p = [0.8182, 0.7144, 0.7483]
p = [0.8182, 0.2468, 0.1320]

pvals = PValues(p)
adjust(pvals, BenjaminiHochberg())

3-element Vector{Float64}:
 0.8182
 0.3702
 0.3702